<a href="https://colab.research.google.com/github/NirantK/Hinglish/blob/BERT/PretrainHinglishBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hinglish BERT
 
Pre-train and Finetune Multilingual BERT for Hinglish twitter data

In [0]:
# Getting Multilingial BERT models. This one is multilingual cased 
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip multi_cased_L-12_H-768_A-12.zip

--2020-01-31 14:58:20--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c10::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M   224MB/s    in 2.8s    

2020-01-31 14:58:23 (224 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-7

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re
def clean(df):
    df["clean_text"] = df["text"]
    df["clean_text"] = (
        (df["clean_text"])
            .apply(lambda text: re.sub(r"RT\s@\w+", "", text)) #Removes RTS
            .apply(lambda text: re.sub(r"@", "mention", text)) # Replaces @ with mention
            .apply(lambda text: re.sub(r"#", "hashtag", text)) # Replaces # with hastag
            .apply(lambda text: re.sub(r"http\S+", "", text)) # Removes URL
        )
    return df

In [0]:
import pandas as pd
import re

# train and test json are available here https://drive.google.com/open?id=1Ij_THj01L-2Y1a8woOn9vXmHosW83e3R
# They are labeled twitter jsons with sentiments ["positive", "negative", "neutral"] attached to them
traindf= pd.read_json("drive/My Drive/Hinglish/interim/train.json")
testdf= pd.read_json("drive/My Drive/Hinglish/interim/test.json")

traindf = clean(traindf)
testdf = clean(testdf)
len(traindf), len(testdf)

(13600, 3400)

In [0]:
df_high = pd.read_csv("drive/My Drive/Hinglish/interim/hinglish_unsup_high_confidence.csv", names=["text"])
df_high = clean(df_high)
len(df_high)

162857

In [0]:
df_less = pd.read_csv("drive/My Drive/Hinglish/interim/hinglish_unsup_less_confidence.csv", names=["text"])
df_less = clean(df_less)
len(df_less)

384319

In [0]:
# Concatinating
textlist = list(df_less['clean_text'].dropna())+list(df_high['clean_text'].dropna())+list(traindf['clean_text'].dropna())+list(testdf['clean_text'].dropna())
len(textlist) 

564176

In [0]:
# ##Uncomment this to save the data created in gooogle drive
# open("drive/My Drive/Hinglish/HinglishBERT/data_lm.txt", "w").close()
# with open('drive/My Drive/Hinglish/HinglishBERT/data_lm.txt', 'w') as f:
#     for item in textlist:
#         f.write("%s\n" % item)

In [0]:
!rm test.txt
with open('test.txt', 'w') as f:
    for item in textlist[:500000]:
        f.write("%s\n" % item)

# Getting BERT Repo

In [0]:
# # # Uncomment if bert_repo isn't cloned
# import sys
# !test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
# if not 'bert_repo' in sys.path:
#   sys.path += ['bert_repo']

In [0]:
!python bert_repo/create_pretraining_data.py  --input_file=./test.txt  --output_file=temp_output  --vocab_file=multi_cased_L-12_H-768_A-12/vocab.txt  --do_lower_case=True  --max_seq_length=128  --max_predictions_per_seq=20  --masked_lm_prob=0.15  --random_seed=12345  --dupe_factor=5



W0131 17:22:37.728068 139772941096832 module_wrapper.py:139] From bert_repo/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0131 17:22:37.728235 139772941096832 module_wrapper.py:139] From bert_repo/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0131 17:22:37.728362 139772941096832 module_wrapper.py:139] From /content/bert_repo/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0131 17:22:38.053399 139772941096832 module_wrapper.py:139] From bert_repo/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0131 17:22:38.054915 139772941096832 module_wrapper.py:139] From bert_repo/create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:*** Rea

In [0]:
!python bert_repo/run_pretraining.py  --input_file=temp_output  --output_dir=pretraining_output  --do_train=True  --do_eval=True  --bert_config_file=multi_cased_L-12_H-768_A-12/bert_config.json  --init_checkpoint=multi_cased_L-12_H-768_A-12/bert_model.ckpt  --train_batch_size=32  --max_seq_length=128  --max_predictions_per_seq=20  --num_train_steps=20  --num_warmup_steps=10  --learning_rate=2e-5 --save_checkpoints_step 100 --iterations_per_loop 100




W0131 17:33:34.346554 140573676734336 module_wrapper.py:139] From bert_repo/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0131 17:33:34.346783 140573676734336 module_wrapper.py:139] From bert_repo/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0131 17:33:34.346968 140573676734336 module_wrapper.py:139] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0131 17:33:34.347652 140573676734336 module_wrapper.py:139] From bert_repo/run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0131 17:33:34.347948 140573676734336 module_wrapper.py:139] From bert_repo/run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0131 17:33:34.348993 140573676734336 module_wrapper.py:139] 

In [0]:
# !mkdir pretrained_bert
# !cp ./pretraining_output/model.ckpt-20.index ./pretrained_bert/bert_model.ckpt.index
# !cp ./pretraining_output/model.ckpt-20.meta ./pretrained_bert/bert_model.ckpt.meta
# !cp ./pretraining_output/model.ckpt-20.data-00000-of-00001 ./pretrained_bert/bert_model.ckpt.data-00000-of-00001
# !cp ./multi_cased_L-12_H-768_A-12/vocab.txt ./pretrained_bert/vocab.txt
# !cp ./multi_cased_L-12_H-768_A-12/bert_config.json ./pretrained_bert/bert_config.json

mkdir: cannot create directory ‘pretrained_bert’: File exists


In [0]:
# !tar cvzf pretrained_bert500k.tar.gz pretrained_bert
# !cp -r pretrained_bert500k.tar.gz drive/My\ Drive/Hinglish/

pretrained_bert/
pretrained_bert/bert_model.ckpt.data-00000-of-00001
pretrained_bert/bert_config.json
pretrained_bert/bert_model.ckpt.meta
pretrained_bert/vocab.txt
pretrained_bert/bert_model.ckpt.index


In [0]:
! pip install keras-bert -q

In [0]:
import os
import json
from keras_bert.loader import load_trained_model_from_checkpoint
from keras_bert.bert import *
BERT_PRETRAINED_DIR = "pretrained_bert"
config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True)

model.summary(line_length=120)

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Model: "model_1"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, 512)                0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, 512)                0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 512, 768), (119547 91812096      Input-Token[0][0]                       
__________